In [5]:
def find_summary_regions_selected(srt_filename, summarizer, duration, language ,bonusWords, stigmaWords, videonamepart):
    srt_file = pysrt.open(srt_filename)
    

    clipList = list(map(srt_item_to_range,srt_file))

    avg_subtitle_duration = total_duration_of_regions(clipList)/len(srt_file)

    
    n_sentences = duration / avg_subtitle_duration
    print("nsentance : "+str(n_sentences))

    
    [summary,summarizedSubtitles] = summarize(srt_file, summarizer, n_sentences, language, bonusWords, stigmaWords)
    
    total_time = total_duration_of_regions(summary)
    print("total_time : "+str(total_time))
    try_higher = total_time < duration
    prev_total_time = -1
    
    if try_higher:
        
        while total_time < duration:
            if(prev_total_time==total_time):
                print("1 : Maximum summarization time reached")
                break
            print("1 : total_time : duration "+str(total_time)+" "+str(duration))
            n_sentences += 1
            [summary,summarizedSubtitles] = summarize(srt_file, summarizer, n_sentences, language, bonusWords, stigmaWords)
            prev_total_time=total_time
            total_time = total_duration_of_regions(summary)
    else:
        while total_time > duration:
            if(n_sentences<=2):
                print("2 : Minimum summarization time reached")
                break
            print("2 : total_time : duration "+str(total_time)+str(duration))
            n_sentences -= 1
            [summary,summarizedSubtitles] = summarize(srt_file, summarizer, n_sentences, language, bonusWords, stigmaWords)
            total_time = total_duration_of_regions(summary)


    path=videonamepart+str(summarizer)+"_summarized.srt"
    with open(path,"w+") as sf:
        for i in range(0,len(summarizedSubtitles)):
            sf.write(str(summarizedSubtitles[i]))
            sf.write("\n")
    sf.close()

    #test file for finding emotions
    # path = "./media/documents/summarizedSubtitleText.txt"
    # with open(path,"w+") as stf:
    #     for i in range(0,len(summarizedSubtitles)):
    #         stf.write(str(summarizedSubtitles[i].text))
    #         stf.write("\n")
    # stf.close()

    # return the resulant summarized subtitle array
    return summary,summarizedSubtitles

def createComVideo(videoName,subtitleName,dummyTxt,summTypes):
    print("Non Weighted Algorithm")
    summarizers=[]
    for item in summTypes:
        if(item):
            print(item)
            summarizers.append(item)

    print(summarizers)
    #summarizers = ["LR","LS","LU","TR"]

    # dstPathVideo = [] #currently not used
    #create all videos
    # for summType in summarizers:
    #     dstPathVideo.append(summarizeVideo(videoName,subtitleName,summType,summTime,dummyTxt,dummyTxt))

    videoTotSubtile=pysrt.open(subtitleName)
    clipList=list(map(srt_item_to_range,videoTotSubtile))
    summTime=total_duration_of_regions(clipList)/1.7 #taking half of subtitle's time of a video

    base, ext = os.path.splitext(videoName)
    print("base : "+str(base))
    videonamepart = "{0}_".format(base)
    #commonName = videonamepart +str(summarizerName)+"_summarized"

    summarizeList=[]
    for summType in summarizers:
        obj=find_summary_regions_selected(subtitleName,summType,int(summTime),'english',dummyTxt,dummyTxt,videonamepart)
        print("------------------")
        print(obj[1])
        print("------------------")
        node=Summary(summType,obj[0],obj[1])
        summarizeList.append(node)

    for summObj in summarizeList:
        makeCorrectTime(summObj,videonamepart)


In [4]:
from moviepy.editor import *
    
# loading video dsa gfg intro video  
clip = VideoFileClip("NFL.mp4")  
     
# getting only first 5 seconds  
clip = clip.subclip(0, 15)  
  
# cutting out some part from the clip 
clip = clip.cutout(3, 10) 
  
# showing  clip  
clip.ipython_display(width = 360)  


Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [3]:
from moviepy.editor import VideoFileClip, TextClip, ImageClip, concatenate_videoclips

In [ ]:
def create_summary(filename, regions):
    subclips = []
    # obtain video
    input_video = VideoFileClip(filename)
    # Scan through video and store the subclips in an array
    last_end = 0
    for (start, end) in regions:
        subclip = input_video.subclip(start, end)
        subclips.append(subclip)
        last_end = end

    # return the concatenated videoclip to the
    return concatenate_videoclips(subclips)